# Code for competition

In [ ]:
#!apt -y install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8

#!pip uninstall -y tensorflow
#!pip uninstall -y tensorflow-transform
#!pip uninstall -y tensorflow-io
#!pip install tensorflow-transform
#!pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import shutil
from collections import Counter
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from PIL import Image
import re
import time
from IPython.display import FileLink

!nvidia-smi

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
print(tf.config.list_physical_devices())

# Enable experimental feature of memory occupation growth control
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for dev in physical_devices:
    tf.config.experimental.set_memory_growth(dev, True)

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Enable distributed training
strategy = tf.distribute.MirroredStrategy()

# Random seed for reproducibility
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)


### Metadata

In [2]:
classes = ["Species1", "Species2", "Species3", "Species4",
           "Species5", "Species6", "Species7", "Species8"]
input_shape = (96, 96, 3)
input_size = input_shape[:-1]
inflation_coeff = 1.5


256


Nvidia SLI GPU optimizations for 2xT4 on kaggle

In [ ]:
# Disable AutoShard
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF


In [3]:
path = os.getcwd()
if not os.path.exists(path+'/training_data_final'):
    shutil.copytree('../input/competition/training_data_final',
                    os.getcwd() + r'/training_data_final')
print(os.listdir(os.getcwd()))


['training_data_final', '.virtual_documents', '__notebook_source__.ipynb']


### Prepare the environment

In [4]:
train_split = 0.8

path = os.getcwd()
if not os.path.exists(path+'/training') and not os.path.exists(path+'/validation'):
    os.mkdir(path+'/training')
    os.mkdir(path+'/validation')

    # Destination path
    dest_train = path + '/training'
    dest_valid = path + '/validation'

    # Source path
    source = path + '/training_data_final'

    # Create train and validation into the training and validation folders
    for folder in os.listdir(source):
        if not os.path.exists(dest_train + '/' + folder):
            os.mkdir(dest_train + '/' + folder)
        if not os.path.exists(dest_valid + '/' + folder):
            os.mkdir(dest_valid + '/' + folder)

        # Create path of the class
        class_source = source + '/' + folder
        # List of files for the class
        files = os.listdir(class_source)
        # Split is performed randomly
        random.shuffle(files)

        # Create training set randomly
        for i in range(int(len(files) * train_split)):
            # Copy an image in the training set
            dest = shutil.copy(
                class_source+'/'+files[i], dest_train+'/'+folder+'/'+files[i])

        # Create validation set randomly
        for j in range(i + 1, len(files)):
            # copy an image in the validation set
            dest = shutil.copy(
                class_source+'/'+files[j], dest_valid+'/'+folder+'/'+files[j])


### Prepare the training set for standardization

In [5]:
from tensorflow.keras.applications.xception import preprocess_input


def preprocessing(image):
    # return tf.image.adjust_saturation(image, 3)
    return preprocess_input(image)
    # return image


In [6]:
samples = []
targets = []

dest_train = os.getcwd() + '/training'

for folder in os.listdir(dest_train):
    dest_class = dest_train + '/' + folder
    i = int(re.sub("\D", "", folder)) - 1
    for img in os.listdir(dest_class):
        temp = Image.open(dest_class + '/' + img).convert('RGB')
        image = preprocessing(np.squeeze(np.expand_dims(temp, axis=0)))
        label = tfk.utils.to_categorical(i, len(classes))
        samples.append(image)
        targets.append(label)
X_train = np.array(samples)
y_train = np.array(targets, dtype=np.uint8)
print(X_train.shape, X_train.dtype, sep=", ")
print(y_train.shape, y_train.dtype, sep=", ")

# Compute the class weights in order to balance loss during training
y_numeric = []
for v in y_train:
    y_numeric.append(np.argmax(v))

labels = np.unique(np.fromiter([np.argmax(t) for t in y_train], np.int32))

class_weights = dict(enumerate(class_weight.compute_class_weight(
    'balanced', classes=labels, y=y_numeric)))
print(class_weights)


(2829, 96, 96, 3), float32
(2829, 8), uint8
{0: 2.389358108108108, 1: 0.8320588235294117, 2: 0.8583131067961165, 3: 0.8667279411764706, 4: 0.8340212264150944, 5: 1.9978813559322033, 6: 0.8243006993006993, 7: 0.8709975369458128}


### Static augmentation (only on training set)

In [7]:
static_aug = True
balanced = False

if static_aug and not os.path.exists(path+'/training_aug'):
    old_train = os.getcwd() + '/training'
    dest_train = os.getcwd() + '/training_aug'
    shutil.copytree(old_train, dest_train)

    desired_amount = int(537 * train_split)

    static_gen = ImageDataGenerator()

    for folder in os.listdir(dest_train):
        dest_path = dest_train + '/' + folder
        label = int(re.sub("\D", "", folder)) - 1

        if balanced:
            to_produce = desired_amount - len(os.listdir(dest_path))
        else:
            class_expansion = [6, 1, 1, 1, 1, 3, 1, 4]
            to_produce = (
                class_expansion[label] * desired_amount) - len(os.listdir(dest_path))

        static_gen_data = static_gen.flow_from_directory(dest_train,
                                                         batch_size=1,
                                                         target_size=input_size,
                                                         classes=[folder],
                                                         class_mode='categorical',      # Targets are directly converted into one-hot vectors
                                                         shuffle=False,
                                                         seed=seed)

        print(f'Computing {to_produce} augmented images for target "{folder}"')
        os.chdir(dest_path)
        for i in range(0, to_produce):
            Image.fromarray(np.squeeze(next(static_gen_data)[0]).astype(
                np.uint8)).save(f'aug{i:05}.jpg')
        os.chdir('../')

    os.chdir('../')
    print('\n' + os.getcwd())


Found 406 images belonging to 1 classes.
Computing 1310 augmented images for target "Species8"
Found 177 images belonging to 1 classes.
Computing 1110 augmented images for target "Species6"
Found 429 images belonging to 1 classes.
Computing 0 augmented images for target "Species7"
Found 425 images belonging to 1 classes.
Computing 4 augmented images for target "Species2"
Found 408 images belonging to 1 classes.
Computing 21 augmented images for target "Species4"
Found 148 images belonging to 1 classes.
Computing 2426 augmented images for target "Species1"
Found 412 images belonging to 1 classes.
Computing 17 augmented images for target "Species3"
Found 424 images belonging to 1 classes.
Computing 5 augmented images for target "Species5"

/kaggle/working


### Online augmentation
Lets create the generators we'll need...

In [ ]:
batch_size = 128 * strategy.num_replicas_in_sync
epochs = 400

print("batch size = ", batch_size)


Generators

In [8]:

train_data_gen = ImageDataGenerator(rotation_range=180,
                                    width_shift_range=30,
                                    height_shift_range=30,
                                    horizontal_flip=True,
                                    brightness_range=[0.8, 1.2],
                                    # channel_shift_range=150,
                                    zoom_range=[0.7, 1.3],
                                    shear_range=0.1,
                                    fill_mode='reflect',
                                    preprocessing_function=preprocessing
                                    )

valid_data_gen = ImageDataGenerator(preprocessing_function=preprocessing)

# Fit the standardization values
# train_data_gen.fit(X_train)
# valid_data_gen.fit(X_train)


... using flow_from_directory

In [9]:
# Setting right paths
path = os.getcwd()
if static_aug:
    training_dir = path + '/training_aug'
else:
    training_dir = path + '/training'
validation_dir = path + '/validation'

# Training
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=batch_size,
                                               target_size=input_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=seed)

# Validation
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=batch_size,
                                               target_size=input_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=seed)

# Create Datasets objects
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(
                                                   tf.float16, tf.uint8),
                                               output_shapes=([None, input_shape[0], input_shape[1], input_shape[2]], [None, len(classes)]))

train_dataset = train_dataset.with_options(options)
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(
                                                   tf.float16, tf.uint8),
                                               output_shapes=([None, input_shape[0], input_shape[1], input_shape[2]], [None, len(classes)]))

valid_dataset = valid_dataset.with_options(options)
valid_dataset = valid_dataset.repeat()


Found 7722 images belonging to 8 classes.
Found 713 images belonging to 8 classes.


### Prepare the validation set for evaluation purposes

In [10]:
samples = []
targets = []

#mean = train_data_gen.mean
#std = train_data_gen.std

dest_valid = os.getcwd() + '/validation'

for folder in os.listdir(dest_valid):
    dest_class = dest_valid + '/' + folder
    i = int(re.sub("\D", "", folder)) - 1
    for img in os.listdir(dest_class):
        temp = Image.open(dest_class + '/' + img).convert('RGB')
        #image = preprocessing((np.squeeze(np.expand_dims(temp, axis=0)) - mean) / std)
        image = preprocessing(np.squeeze(np.expand_dims(temp, axis=0)))
        label = tfk.utils.to_categorical(i, len(classes))
        samples.append(image)
        targets.append(label)

X_val = np.array(samples, dtype=np.float16)
y_val = np.array(targets, dtype=np.uint8)
print(X_val.shape, X_val.dtype, sep=", ")
print(y_val.shape, y_val.dtype, sep=", ")


(713, 96, 96, 3), float16
(713, 8), uint8


### Models definition functions and utilities for model evaluation

In [ ]:
'''
decay_rate = 5  # patience should be set: changes_to_see * decay_rate + 1
min_lr = 2e-5
def scheduler(epoch, lr):
    if epoch % decay_rate == (decay_rate - 1):
        return max(lr * tf.math.exp(-0.1), min_lr)
    return lr
'''

# learning rate scheduler for fine tuning
initial_lr = 1e-4
def scheduler(epoch, lr):
    if epoch <= 10:
        return initial_lr
    else:
        return max(initial_lr * tf.math.exp(- (epoch-10) / 30.0), 5e-6)

lr_scheduler = tfk.callbacks.LearningRateScheduler(scheduler)


def get_lr_metric():
    def lr(y_true, y_pred):
        # I use ._decayed_lr method instead of .lr
        return tfk.optimizers.Adam()._decayed_lr(tf.float32)
    return lr


lr_metric = get_lr_metric()

early_stop = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)

'''
start = time.time()
class ElapsedTimeCallback(tfk.callbacks.Callback):
    def on_test_end(self, epoch, logs=None):
        el = time.time() - start
        print(f'\nElapsed time: {int(el // 60)} minutes {(el % 60):.3f} seconds')
'''


In [ ]:
def plot_history(history):
    plt.figure(figsize=(15, 5))
    plt.plot(history['loss'], label='Std training',
             alpha=.3, color='#ff7f0e', linestyle='--')
    plt.plot(history['val_loss'], label='Std validation',
             alpha=.8, color='#ff7f0e')
    plt.legend(loc='upper left')
    plt.title('Categorical Crossentropy')
    plt.grid(alpha=.3)

    plt.figure(figsize=(15, 5))
    plt.plot(history['accuracy'], label='Std training',
             alpha=.8, color='#ff7f0e', linestyle='--')
    plt.plot(history['val_accuracy'], label='Std validation',
             alpha=.8, color='#ff7f0e')
    plt.legend(loc='upper right')
    plt.title('Accuracy')
    plt.grid(alpha=.3)

    plt.show()


In [ ]:
def score_model(model):
    predictions = model.predict(X_val)
    cm = confusion_matrix(np.argmax(y_val, axis=-1),
                          np.argmax(predictions, axis=-1))

    accuracy = accuracy_score(
        np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1))
    precision = precision_score(
        np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    recall = recall_score(np.argmax(y_val, axis=-1),
                          np.argmax(predictions, axis=-1), average='macro')
    f1 = f1_score(np.argmax(y_val, axis=-1),
                  np.argmax(predictions, axis=-1), average=None)
    print('Accuracy:', accuracy.round(4))
    print('Precision:', precision.round(4))
    print('Recall:', recall.round(4))
    print('F1:', f1.round(4))

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm.T, xticklabels=classes, yticklabels=classes)
    plt.xlabel('True labels')
    plt.ylabel('Predicted labels')
    plt.show()


### Models definitions

In [11]:
def tune_supernet_v1(input_shape):
    tf.random.set_seed(seed)

    # Load the supernet
    supernet = tfk.applications.Xception(
        include_top=False, input_shape=(192, 192, 3))
    supernet.set_weights(tfk.models.load_model(
        'supernet_xception_v2').get_layer('xception').get_weights())
    # Use the supernet only as feature extractor
    supernet.trainable = True  # "True" for fine tuning
    for layer in supernet.layers[:-15]:
        layer.trainable = False

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    x = tfkl.Resizing(192, 192, interpolation="bicubic",
                      name='resizing')(input_layer)

    x = supernet(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)

    x_gap = x

    x = tfkl.Dropout(0.3, seed=seed, name='dropout')(x)

    x = tfkl.Dense(
        units=2048,
        activation='relu',
        kernel_initializer=tfk.initializers.HeUniform(seed),
        name='classifier')(x)

    # Skip connection
    x = tfkl.Add(name='adder')([x_gap, x])

    #x = tfkl.Dropout(0.3, seed=seed, name='dropout')(x)

    output_layer = tfkl.Dense(
        units=len(classes),
        activation='softmax',
        kernel_initializer=tfk.initializers.GlorotUniform(seed),
        name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer,
                      outputs=output_layer, name='tuned_model_v1')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(
    ), optimizer=tfk.optimizers.Adam(), metrics=['accuracy', get_lr_metric()])

    # Return the model
    return model


### Train the model

In [ ]:
with strategy.scope():
    model_v1 = tune_supernet_v1(input_shape)
    model_v1.summary()


In [ ]:

history = model_v1.fit(x=train_dataset,
                       # Only indicative since we set "repeat" in training and validation datasets
                       epochs=epochs,
                       steps_per_epoch=int(len(train_gen) * 1.5),
                       validation_data=valid_dataset,
                       validation_steps=len(valid_gen),
                       class_weight=class_weights,
                       callbacks=[early_stop, lr_scheduler]
                       ).history

# ElapsedTimeCallback()


Save the tuned model

In [14]:
model_v1.save('xception_tuned_v1', include_optimizer=False)

shutil.make_archive('xception_tuned_v1', 'zip', 'xception_tuned_v1')
FileLink(r'xception_tuned_v1.zip')


/kaggle/working/best_model.zip

Model evaluation and training history plots

In [ ]:
plot_history(history)
score_model(model_v1)
